## Imports

In [ ]:
!which python
# !pip install efficientnet_pytorch

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
from pathlib import Path
from scipy.special import softmax
from functools import partial
import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, roc_auc_score

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import torch
import torch.utils.data as Data
import torch.nn as nn

from transformers import get_cosine_schedule_with_warmup
from torchvision import transforms as T, models
from torchsummary import summary

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from src.plant_pathology.leaf_dataset import LeafDataset
from src.plant_pathology.model_loops import training, validation, testing
from src.plant_pathology.models import get_resnet, get_densenet, get_effecientnet
from src.plant_pathology.visualizations import show_saliency_maps, create_class_visualization
from src.plant_pathology.models import initialize_model

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Input

In [ ]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
    torch.cuda.empty_cache()
else:
    device = torch.device('cpu')


print('using device:', device)

In [ ]:
# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure
data_dir = "./plant-pathology-2020-fgvc7/"

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception, effecientnet]
model_name = "effecientnet"

# Number of classes in the dataset
num_classes = 4

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 2

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = False

## Reading/Processing the Data

In [ ]:
IMAGE_PATH = Path(data_dir +'images')

def image_path(file_stem):
    return IMAGE_PATH/f'{file_stem}.jpg'

In [ ]:
train_df = pd.read_csv(data_dir+'train.csv')
test_df = pd.read_csv(data_dir + 'test.csv')

train_paths = train_df['img_file'] = train_df['image_id'].apply(image_path)
test_paths = test_df['img_file'] = test_df['image_id'].apply(image_path)

train_labels = train_df[['healthy','multiple_diseases','rust','scab']]


In [ ]:
train_paths, valid_paths, train_labels, valid_labels = train_test_split(
    train_paths, train_labels, test_size = 0.2, random_state=23, stratify = train_labels)
train_paths.reset_index(drop=True,inplace=True)
train_labels.reset_index(drop=True,inplace=True)
valid_paths.reset_index(drop=True,inplace=True)
valid_labels.reset_index(drop=True,inplace=True)

In [ ]:
train_labels.sum(axis=0)

### Visualize Data

In [ ]:
import cv2
import imutils
import matplotlib
matplotlib.rcParams.update({'font.size': 22})

In [ ]:
scab = cv2.imread(str(train_df.iloc[0]['img_file']))
multi = cv2.imread(str(train_df.iloc[1]['img_file']))
rust = cv2.imread(str(train_df.iloc[1819]['img_file']))
healthy = cv2.imread(str(train_df.iloc[4]['img_file']))
kernel = np.ones((6,6),np.float32)/25

types = [healthy, multi, rust, scab]

In [ ]:
fig, axs = plt.subplots(4, 5)
y_labels = ['Healthy', 'Multi', 'Rust', 'Scab']
x_labels = ['Normal', 'Horizontal Flip', 'Vertical Flip', 'Rotated 25', 'Filtered']

for i in range(4):
    axs[i, 0].imshow(types[i])
    axs[i, 0].set(ylabel=y_labels[i])
    axs[i, 1].imshow(cv2.flip(types[i], 1))
    axs[i, 2].imshow(cv2.flip(types[i], 0))
    axs[i, 3].imshow(imutils.rotate(types[i], 25))
    axs[i, 4].imshow(cv2.filter2D(types[i],-1,kernel))
    
    if (i + 1) == 4:
        for j in range(5):
            axs[i, j].set(xlabel=x_labels[j])
    

for ax in axs.flat:
    ax.set_xticks([])
    ax.set_yticks([])

        
fig.set_size_inches(18.5, 10.5)
fig.savefig('example.jpg')

### Initialization

In [ ]:
TRAIN_SIZE = train_labels.shape[0]
VALID_SIZE = valid_labels.shape[0]

In [ ]:
train_dataset = LeafDataset(train_paths, train_labels)
trainloader = Data.DataLoader(train_dataset, shuffle=True, batch_size = batch_size, num_workers = 2)

valid_dataset = LeafDataset(valid_paths, valid_labels, train = False)
validloader = Data.DataLoader(valid_dataset, shuffle=False, batch_size = batch_size, num_workers = 2)

test_dataset = LeafDataset(test_paths,train = False, test = True)
testloader = Data.DataLoader(test_dataset, shuffle=False, batch_size = batch_size, num_workers = 2)

### Metrics

In [ ]:
weighted_f1 = partial(f1_score, labels=[0, 1, 2, 3], average='weighted')
macro_f1 = partial(f1_score, labels=[0, 1, 2, 3], average='macro')
micro_f1 = partial(f1_score, labels=[0, 1, 2, 3], average='micro')
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
# micro_roc_auc_score = partial(roc_auc_score, labels=[0, 1, 2, 3], average='macro', multi_class='ovr')
# acc_fns = [accuracy_score, weighted_f1, macro_f1, micro_f1, roc_auc_score]
acc_fns = [accuracy_score, weighted_f1, macro_f1, micro_f1]

# Get models

In [ ]:
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
# print(model_ft)

In [ ]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")

if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
# optimizer_ft = torch.optim.SGD(params_to_update, lr=0.001, momentum=0.9)

In [ ]:
optimizer_ft = torch.optim.Adam(model_ft.parameters(), lr=8e-4, weight_decay = 1e-3)
num_train_steps = int(len(train_dataset) / batch_size * num_epochs)
scheduler = get_cosine_schedule_with_warmup(
    optimizer_ft, num_warmup_steps=len(train_dataset)/batch_size*5, num_training_steps=num_train_steps)

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()

### Training loop

In [ ]:
since = time.time()
train_loss = []
valid_loss = []
train_acc = []
val_acc = []

for epoch in range(num_epochs):
    
    tl, ta = training(model_ft, trainloader, optimizer_ft, scheduler, loss_fn, acc_fns, device, TRAIN_SIZE)
    vl, va, conf_mat = validation(model_ft, validloader, loss_fn, acc_fns, confusion_matrix, device, VALID_SIZE)
    train_loss.append(tl)
    valid_loss.append(vl)
    train_acc.append(ta)
    val_acc.append(va)
    
    if (epoch+1)%10==0:
        checkpoint = Path('model_checkpoints/')
        checkpoint.mkdir(exist_ok=True)
        torch.save(model.state_dict(), checkpoint/f'{model.__class__.__name__}_epoch_{epoch}.pt')
    
    printstr = f'Epoch: {epoch} , Train loss: {round(tl, 3)}, Val loss: {round(vl, 3)}, Train accs: {[round(t_a, 3) for t_a in ta]}, Val accs: {[round(v_a, 3) for v_a in va]}'
    tqdm.write(printstr)
    
    
time_elapsed = time.time() - since

print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

In [ ]:
train_acc = np.array(train_acc)
val_acc = np.array(val_acc)

### Save Results for future use

In [ ]:
train_results = pd.DataFrame(train_acc)
train_results[4] = train_loss
train_results.columns = ['Accuracy', 'F1 Weighted', 'F1 Macro', 'F1 Micro', 'Loss']
train_results.index.name = 'Epoch'

val_results = pd.DataFrame(val_acc)
val_results[4] = valid_loss
val_results.columns = ['Accuracy', 'F1 Weighted', 'F1 Macro', 'F1 Micro', 'Loss']
val_results.index.name = 'Epoch'

In [ ]:
train_results.to_csv(f'model_results/{model_ft.__class__.__name__}_train_results.csv')
val_results.to_csv(f'model_results/{model_ft.__class__.__name__}_valid_results.csv')

### Plots

In [ ]:
plt.figure(figsize=(14, 8))
plt.ylim(0,1.5)
sns.lineplot(list(range(len(train_loss))), train_loss)
sns.lineplot(list(range(len(valid_loss))), valid_loss)
plt.xlabel('Epoch', fontsize=18)
plt.ylabel('Loss', fontsize=18)
plt.legend(['Train','Val'], fontsize=16)
plt.title('Loss vs Epoch', fontsize=20)
plt.savefig(f'{model_ft.__class__.__name__}_loss.jpg')

In [ ]:
acc_names = ['Accuracy', 'F1 Weighted', 'F1 Macro', 'F1 Micro']
for idx, acc_name in enumerate(acc_names):
    plt.figure(figsize=(14, 8))
    sns.lineplot(list(range(len(train_acc[:, idx]))), train_acc[:, idx])
    sns.lineplot(list(range(len(val_acc[:, idx]))), val_acc[:, idx])
    plt.xlabel('Epoch', fontsize=18)
    plt.ylabel(acc_name, fontsize=18)
    plt.legend(['Train','Val'], fontsize=16)
    plt.title(f'{acc_name} vs Epoch', fontsize=20)
    plt.savefig(f'{model_ft.__class__.__name__}_accuracy.jpg')

In [ ]:
labels = ['Healthy', 'Multiple','Rust','Scab']
plt.figure(figsize=(15, 10))
sns.heatmap(conf_mat, xticklabels=labels, yticklabels=labels, annot=True)
plt.title('Confusion Matrix', fontsize=20)
plt.savefig(f'{model_ft.__class__.__name__}_confusion.jpg')

### Testing Performance

In [ ]:
def get_testing_output(model, device):
    model = model.to(device)
    subs = []
    for i in range(5): #average over 5 runs
        out = testing(model, testloader, device)
        output = pd.DataFrame(softmax(out,1), columns = ['healthy','multiple_diseases','rust','scab']) #the submission expects probability scores for each class
        output.drop(0, inplace = True)
        output.reset_index(drop=True,inplace=True)
        subs.append(output)

    sub_eff1 = sum(subs)/5
    return sub_eff1

### Model Emsembling

In [ ]:
densenet_checkpoint = get_densenet(train_labels, model_path='./model_checkpoints/DenseNet_epoch_29.pt')
resnet_checkpoint = get_resnet(train_labels, model_path='./model_checkpoints/ResNet_epoch_29.pt')

In [ ]:
sub_densenet = get_testing_output(densenet_checkpoint, device)
sub_resnet = get_testing_output(resnet_checkpoint, device)

In [ ]:
submission = (sub_densenet + sub_resnet)/2

In [ ]:
submission['image_id'] = test_df['image_id']

In [ ]:
submission.to_csv('submission.csv', index=False)

### Saliency Maps

In [ ]:
model = get_densenet(train_labels, model_path='./model_checkpoints/DenseNet_epoch_29.pt')

In [ ]:
class_names = ['Healthy', 'Multi', 'Rust', 'Scab']
y = [0, 1, 2, 3]
X = np.array([
    np.array(Image.open(str(train_df.iloc[1817]['img_file']))),
    np.array(Image.open(str(train_df.iloc[1]['img_file']))),
    np.array(Image.open(str(train_df.iloc[1819]['img_file']))),
    np.array(Image.open(str(train_df.iloc[0]['img_file']))),
])

In [ ]:
show_saliency_maps(X, y, model, class_names, device)

### Class Visualization

In [ ]:
model = get_densenet(train_labels, model_path='./model_checkpoints/DenseNet_epoch_29.pt')

In [ ]:
create_class_visualization(y[3], model, device, class_names)